<a href="https://colab.research.google.com/github/KN196883/bond_pricer/blob/master/FIXED_BOND_PRICER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Python class and jupyter iPython notebook for pricing a fixed coupon bond

In [152]:
#Githubのcsvをurlから読み込み

import pandas as pd
import urllib.request
from io import StringIO

url = 'https://raw.githubusercontent.com/KN196883/bond_pricer/master/usbondall2.csv'

#csvを読み込む関数
def read_csv(url):
    print(url)
    res = urllib.request.urlopen(url)
    res = res.read().decode("utf-8")
    df = pd.read_csv(StringIO( res) )
    return df

#実行
df=read_csv(url)
df=df.drop(['発行体名','Unnamed: 16'],axis=1)

https://raw.githubusercontent.com/KN196883/bond_pricer/master/usbondall2.csv


In [153]:
import pandas as pd
import numpy as np
import math as m
import datetime as dt
from dateutil.relativedelta import relativedelta
import calendar

##################################################################################
#Adding daycount convention functions
##################################################################################
def day_count_actual_360(start_date, end_date):
    #Returns number of days between start_date and end_date, using Actual/360 convention
    return (end_date - start_date).days

def day_count_actual_365(start_date, end_date):
    #Returns number of days between start_date and end_date, using Actual/365 convention
    return (end_date - start_date).days

def day_count_actual_actual(start_date, end_date):
    #Returns number of days between start_date and end_date, using Actual/Actual convention
    return (end_date - start_date).days

def day_count_30_360(start_date, end_date):
    #Returns number of days between start_date and end_date, using Thirty/360 convention
    d1 = min(30, start_date.day)
    d2 = min(d1, end_date.day) if d1 == 30 else end_date.day
    return 360*(end_date.year - start_date.year) + 30*(end_date.month - start_date.month) + d2 - d1

def day_count_30E_360(start_date, end_date):
    #Returns number of days between start_date and end_date, using ThirtyE/360 convention
    d1 = min(30, start_date.day)
    d2 = min(30, end_date.day)
    return 360 * (end_date.year - start_date.year) + 30 * (end_date.month - start_date.month) + d2 - d1





#######################################################################
#月末日計算(2/28応答日用)
#######################################################################

def get_last_date(dt):
    return dt.replace(day=calendar.monthrange(dt.year, dt.month)[1])




  

class fixed_bond:
    def __init__(self, issue_date, settlement_date, first_coupon_date, maturity_date, face_value, reoffer_yield, coupon_rate, first_coupon_type, coupon_frequency, day_count_convention, redemption_rate):
        ##################################################################################
        #Setting up inital attributes
        ##################################################################################
        self.issue_date = issue_date
        self.settlement_date = settlement_date
        self.first_coupon_date = first_coupon_date
        self.maturity_date = maturity_date
        self.face_value = face_value
        self.reoffer_yield = reoffer_yield
        self.coupon_rate = coupon_rate
        self.first_coupon_type = first_coupon_type
        self.coupon_frequency = coupon_frequency
        self.day_count_convention = day_count_convention
        self.redemption_rate = redemption_rate

        ##################################################################################
        #Parsing input values
        ##################################################################################
        #Converting reoffer yield to percentage
        self.reoffer_yield = self.reoffer_yield/100

        #Converting coupon rate to percentage
        self.coupon_rate = self.coupon_rate/100

        #parsing coupon frequency
        if self.coupon_frequency == "ANNUAL":
            self.coupon_frequency_modifier = 1
        elif self.coupon_frequency == "SEMI-ANNUAL":
            self.coupon_frequency_modifier = 2
        elif self.coupon_frequency == "QUARTERLY":
            self.coupon_frequency_modifier = 4
        elif self.coupon_frequency_modifier == "MONTHLY":
            self.coupon_frequency_modifier = 12
        else:
            self.coupon_frequency_modifier = 1
            #need to add error handling

        #Converting redemption rate to percentage
        self.redemption_rate = self.redemption_rate/100

        #Generating accrual start date
        time_modifier = int(12/self.coupon_frequency_modifier)

        # if self.first_coupon_type == "REGULAR":
        #     self.accrual_start_date = self.settlement_date
        # else:
        #     self.accrual_start_date = self.first_coupon_date - relativedelta(months=time_modifier)

        self.accrual_start_date=self.first_coupon_date-relativedelta(months=time_modifier)


        ##################################################################################
        #Generating the columns for the cash flow table
        ##################################################################################
        #generating cash flow schedule
        cash_flow_date = self.first_coupon_date
        cash_flow_dates = [cash_flow_date]
        self.number_of_periods = 1


        ######ここから
        if cash_flow_date==get_last_date(cash_flow_date):
          eom=1
        else:
          eom=0        
      

        while cash_flow_date < maturity_date:
            cash_flow_date += relativedelta(months=time_modifier)
            if eom==1:
              cash_flow_date=get_last_date(cash_flow_date)

            cash_flow_dates.append(cash_flow_date)
            self.number_of_periods += 1

        self.cash_flow_dates = cash_flow_dates

      #######ここ


        #computing cash flow periods
        cash_flow_periods = []
        for i in range(self.number_of_periods):
            cash_flow_periods.append(i + 1)
        self.cash_flow_periods = cash_flow_periods

        # computing future cash flows per date
        self.coupon_payment = (self.face_value*self.coupon_rate)/self.coupon_frequency_modifier
        self.principal_payment = self.face_value * self.redemption_rate

        cash_flow_amounts = []
        for i in range(self.number_of_periods):
            if i == self.number_of_periods - 1:
                cash_flow = self.principal_payment + self.coupon_payment
            else:
                cash_flow = self.coupon_payment

            cash_flow_amounts.append(cash_flow)
        self.cash_flow_amounts = cash_flow_amounts

        #Computing discount rate
        self.discount_rate = 1 + (self.reoffer_yield/self.coupon_frequency_modifier)
        self.discount_rates = []
        for i in range(self.number_of_periods):
            self.discount_rates.append(self.discount_rate)

        #Computing discount period
        if self.first_coupon_type == "REGULAR":
            self.discount_period = 1
            accrued_numerator = day_count_actual_actual(self.accrual_start_date,self.settlement_date)
            denominator = day_count_actual_actual(self.accrual_start_date,self.first_coupon_date)
            accrued_denominator = denominator
            self.accrued_period = accrued_numerator/accrued_denominator
            self.accrued_days = accrued_numerator        
            self.accrued_interest = self.cash_flow_amounts[0]*self.accrued_period



        else:
            if self.day_count_convention == "ACTUAL/ACTUAL":
                numerator = day_count_actual_actual(self.settlement_date,self.first_coupon_date)
                denominator = day_count_actual_actual(self.accrual_start_date,self.first_coupon_date)
                self.discount_period = numerator/denominator
                if self.first_coupon_type == "FULL SHORT FIRST":
                    accrued_numerator = day_count_actual_actual(self.accrual_start_date,self.settlement_date)
                    accrued_denominator = denominator
                    self.accrued_period = accrued_numerator/accrued_denominator
                    self.accrued_days = accrued_numerator
                    self.accrued_interest = self.cash_flow_amounts[0]*self.accrued_period
                else:
                    self.accrued_period = 0
                    self.accrued_days = 0
                    self.accrued_interest = 0

            elif self.day_count_actual_365 == "ACTUAL/365":
                numerator = day_count_actual_365(self.settlement_date,self.first_coupon_date)
                denominator = 365/self.coupon_frequency_modifier
                self.discount_period = numerator/denominator
                if self.first_coupon_type == "FULL SHORT FIRST":
                    accrued_numerator = day_count_actual_actual(self.accrual_start_date,self.settlement_date)
                    accrued_denominator = denominator
                    self.accrued_period = accrued_numerator/accrued_denominator
                    self.accrued_days = accrued_numerator
                    self.accrued_interest = self.cash_flow_amounts[0]*self.accrued_period
                else:
                    self.accrued_period = 0
                    self.accrued_days = 0
                    self.accrued_interest = 0

            elif self.day_count_actual_360 == "ACTUAL/360":
                numerator = day_count_actual_360(self.settlement_date,self.first_coupon_date)
                denominator = 360/self.coupon_frequency_modifier
                self.discount_period = numerator/denominator
                if self.first_coupon_type == "FULL SHORT FIRST":
                    accrued_numerator = day_count_actual_actual(self.accrual_start_date,self.settlement_date)
                    accrued_denominator = denominator
                    self.accrued_period = accrued_numerator/accrued_denominator
                    self.accrued_days = accrued_numerator
                    self.accrued_interest = self.cash_flow_amounts[0]*self.accrued_period
                else:
                    self.accrued_period = 0
                    self.accrued_days = 0
                    self.accrued_interest = 0

            elif self.day_count_30_360 == "30/360":
                numerator = day_count_30_360(self.settlement_date,self.first_coupon_date)
                denominator = 360/coupon_frequency_modifier
                self.discount_period = numerator/denominator
                if self.first_coupon_type == "FULL SHORT FIRST":
                    accrued_numerator = day_count_actual_actual(self.accrual_start_date,self.settlement_date)
                    accrued_denominator = denominator
                    self.accrued_period = accrued_numerator/accrued_denominator
                    self.accrued_days = accrued_numerator
                    self.accrued_interest = self.cash_flow_amounts[0]*self.accrued_period
                else:
                    self.accrued_period = 0
                    self.accrued_days = 0
                    self.accrued_interest = 0

            elif self.day_count_30E_360 == "30E/360":
                numerator = day_count_30E_360(self.settlement_date,self.first_coupon_date)
                denominator = 360/coupon_frequency_modifier
                self.discount_period = numerator/denominator
                if self.first_coupon_type == "FULL SHORT FIRST":
                    accrued_numerator = day_count_actual_actual(self.accrual_start_date,self.settlement_date)
                    accrued_denominator = denominator
                    self.accrued_period = accrued_numerator/accrued_denominator
                    self.accrued_days = accrued_numerator
                    self.accrued_interest = self.cash_flow_amounts[0]*self.accrued_period
                else:
                    self.accrued_period = 0
                    self.accrued_days = 0
                    self.accrued_interest = 0
        
        self.discount_periods = []
        for i in range(self.number_of_periods):
            self.discount_periods.append(self.discount_period+i)

        #Adjusting first coupon if it's a full short first
        if self.first_coupon_type == "FULL SHORT FIRST":
            self.cash_flow_amounts[0] = self.coupon_payment
        elif self.first_coupon_type != "REGULAR":
            self.cash_flow_amounts[0] = self.coupon_payment * self.discount_period

        #generating a cash flow table and calculating reoffer cash price
        #table - coupon number / coupon date / cash flow / discount rate / discount period / npv
        df = pd.DataFrame(np.column_stack([self.cash_flow_periods, self.cash_flow_dates, self.cash_flow_amounts, self.discount_rates, self.discount_periods]), 
                               columns=['coupon number', 'coupon_date', 'cash_flow', 'discount_rate', 'discount_period'])
        df['npv'] = (df.cash_flow / (df.discount_rate**df.discount_period))

        self.cash_flow_table = df
        self.reoffer_price = (self.cash_flow_table.npv.sum()/self.face_value)*100
        self.dirty_price = self.reoffer_price + (self.accrued_interest/self.face_value)

In [154]:
##################################################################################
#Using the class
##################################################################################
issue_date = dt.date(2021,2,16)
settlement_date = dt.date(2021,2,26)
first_coupon_date = dt.date(2021,8,15)
maturity_date = dt.date(2031,2,15)
face_value = 1000
reoffer_yield = 1.399
coupon_rate = 1.125
first_coupon_type = "REGULAR"
coupon_frequency = "SEMI-ANNUAL"
day_count_convention = "ACTUAL/ACTUAL"
redemption_rate = 100

test_bond = fixed_bond(issue_date,settlement_date,first_coupon_date, maturity_date,face_value,reoffer_yield,coupon_rate,first_coupon_type,coupon_frequency,day_count_convention,redemption_rate)

print(test_bond.cash_flow_table)
print(f"Reoffer clean price is: {round(test_bond.reoffer_price,3)}%")
print(f"Days of accrued interest: {test_bond.accrued_days}")
print(f"Reoffer dirty price is: {round(test_bond.dirty_price,3)}%")

##################################################################################
#List of inputs
##################################################################################
#issue_date, settlement_date, first_coupon_date,maturity_date = datetime objects (yyyy,m,d)
#face_value = float
#reoffer_yield = for example, a reoffer yield of 2.3% is entered as 2.3
#coupon_rate = for example, a coupon of 2.25% is entered as 2.25
#first_coupon_type = string: REGULAR, ODD, FULL SHORT FIRST
#coupon_frequency = string: ANNUAL, SEMI-ANNUAL, QUARTERLY, MONTHLY
#day_count_convention = string: ACTUAL/ACTUAL, ACTUAL/365, ACTUAL/360, 30/360, 30E/360
#redemption_rate = for example, a redemption rate of 100% is entered as 100

   coupon number coupon_date cash_flow discount_rate discount_period      npv
0              1  2021-08-15     5.625         1.007               1  5.58593
1              2  2022-02-15     5.625         1.007               2  5.54712
2              3  2022-08-15     5.625         1.007               3  5.50859
3              4  2023-02-15     5.625         1.007               4  5.47033
4              5  2023-08-15     5.625         1.007               5  5.43233
5              6  2024-02-15     5.625         1.007               6  5.39459
6              7  2024-08-15     5.625         1.007               7  5.35712
7              8  2025-02-15     5.625         1.007               8  5.31991
8              9  2025-08-15     5.625         1.007               9  5.28295
9             10  2026-02-15     5.625         1.007              10  5.24625
10            11  2026-08-15     5.625         1.007              11  5.20981
11            12  2027-02-15     5.625         1.007            

In [155]:
##################################################################################
#Using the class
##################################################################################
issue_date = dt.date(2021,2,16)
settlement_date = dt.date(2021,2,26)
first_coupon_date = dt.date(2021,3,31)
maturity_date = dt.date(2021,3,31)
face_value = 1000
reoffer_yield = 1.399
coupon_rate = 1.125
first_coupon_type = "REGULAR"
coupon_frequency = "SEMI-ANNUAL"
day_count_convention = "ACTUAL/ACTUAL"
redemption_rate = 100

test_bond = fixed_bond(issue_date,settlement_date,first_coupon_date, maturity_date,face_value,reoffer_yield,coupon_rate,first_coupon_type,coupon_frequency,day_count_convention,redemption_rate)

print(test_bond.cash_flow_table)
print(f"Reoffer clean price is: {round(test_bond.reoffer_price,3)}%")
print(f"Days of accrued interest: {test_bond.accrued_days}")
print(f"Reoffer dirty price is: {round(test_bond.dirty_price,3)}%")

##################################################################################
#List of inputs
##################################################################################
#issue_date, settlement_date, first_coupon_date,maturity_date = datetime objects (yyyy,m,d)
#face_value = float
#reoffer_yield = for example, a reoffer yield of 2.3% is entered as 2.3
#coupon_rate = for example, a coupon of 2.25% is entered as 2.25
#first_coupon_type = string: REGULAR, ODD, FULL SHORT FIRST
#coupon_frequency = string: ANNUAL, SEMI-ANNUAL, QUARTERLY, MONTHLY
#day_count_convention = string: ACTUAL/ACTUAL, ACTUAL/365, ACTUAL/360, 30/360, 30E/360
#redemption_rate = for example, a redemption rate of 100% is entered as 100

  coupon number coupon_date cash_flow discount_rate discount_period     npv
0             1  2021-03-31   1005.62         1.007               1  998.64
Reoffer clean price is: 99.864%
Days of accrued interest: 149
Reoffer dirty price is: 100.687%


In [156]:
test_bond.number_of_periods

1

In [160]:

df_t=df[df['ﾃｨｯｶｰ']=='T'].sort_values('f_maturity').reset_index(drop=True)
tran_list=['満期','初回利払日','前回利払日','次回利払日','発行日','f_maturity']

for i in tran_list:
  df_t[i]=pd.to_datetime(df_t[i])

df_t

,ﾃｨｯｶｰ,ｸｰﾎﾟﾝ,満期,償還ﾀｲﾌﾟ,通貨,CUSIP,ISINｺｰﾄﾞ,初回利払日,前回利払日,日数計算,次回利払日,発行体,発行日,発行額,経過利子,f_cusip,f_name,f_coupon,f_maturity,bval_p,bval_y
0,T,2,2021-02-28,NORMAL,USD,912828B90,US912828B907,2014-08-31,2020-08-31,ACT/ACT,2021-02-28,US TREASURY N/B,2014-02-28,2.908200e+10,9889.5,912828B90 Govt,T 2 02/28/21,2.000,2021-02-28,100.000000,1.000000e-07
1,T,2.5,2021-02-28,NORMAL,USD,9128286D7,US9128286D73,2019-08-31,2020-08-31,ACT/ACT,2021-02-28,US TREASURY N/B,2019-02-28,4.192400e+10,12361.88,9128286D7 Govt,T 2 ½ 02/28/21,2.500,2021-02-28,100.000000,1.000000e-07
2,T,1.125,2021-02-28,NORMAL,USD,912828P87,US912828P873,2016-08-31,2020-08-31,ACT/ACT,2021-02-28,US TREASURY N/B,2016-02-29,4.539500e+10,5562.85,912828P87 Govt,T 1 ⅛ 02/28/21,1.125,2021-02-28,100.000000,1.000000e-07
3,T,2.375,2021-03-15,NORMAL,USD,9128284B3,US9128284B36,2018-09-15,2020-09-15,ACT/ACT,2021-03-15,US TREASURY N/B,2018-03-15,2.800100e+10,10759.67,9128284B3 Govt,T 2 ⅜ 03/15/21,2.375,2021-03-15,100.117188,7.384841e-02
4,T,2.25,2021-03-31,NORMAL,USD,912828C57,US912828C574,2014-09-30,2020-09-30,ACT/ACT,2021-03-31,US TREASURY N/B,2014-03-31,6.900000e+10,9210.16,912828C57 Govt,米国中期･長期国債債 #,2.250,2021-03-31,100.203125,1.177693e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,T,2,2050-02-15,NORMAL,USD,912810SL3,US912810SL35,2020-08-15,2021-02-15,ACT/ACT,2021-08-15,US TREASURY N/B,2020-02-18,6.052100e+10,607.73,912810SL3 Govt,T 2 02/15/50,2.000,2050-02-15,94.179688,2.277563e+00
315,T,1.25,2050-05-15,NORMAL,USD,912810SN9,US912810SN90,2020-11-15,2020-11-15,ACT/ACT,2021-05-15,US TREASURY N/B,2020-05-15,7.357200e+10,3556.63,912810SN9 Govt,T 1 ¼ 05/15/50,1.250,2050-05-15,77.687500,2.305286e+00
316,T,1.375,2050-08-15,NORMAL,USD,912810SP4,US912810SP49,2021-02-15,2021-02-15,ACT/ACT,2021-08-15,US TREASURY N/B,2020-08-17,8.905300e+10,417.82,912810SP4 Govt,T 1 ⅜ 08/15/50,1.375,2050-08-15,80.320312,2.299351e+00
317,T,1.625,2050-11-15,NORMAL,USD,912810SS8,US912810SS87,2021-05-15,2020-11-15,ACT/ACT,2021-05-15,US TREASURY N/B,2020-11-16,8.583800e+10,4623.62,912810SS8 Govt,T 1 ⅝ 11/15/50,1.625,2050-11-15,85.710938,2.291466e+00


In [171]:
list_mat=set(df_t['f_maturity'])

CUSIP_list=[]

for i in list_mat:
  temp_list_mat=df_t[df_t['f_maturity']==i]['発行日']
  new_issue=max(temp_list_mat)
  cusip=df_t[(df_t['f_maturity']==i) & (df_t['発行日']==new_issue)]['CUSIP'].values
  CUSIP_list.append(cusip)



[array(['912810PW2'], dtype=object),
 array(['912828YX2'], dtype=object),
 array(['912810QY7'], dtype=object),
 array(['912810RS9'], dtype=object),
 array(['912810QZ4'], dtype=object),
 array(['91282CBA8'], dtype=object),
 array(['9128286S4'], dtype=object),
 array(['912810SH2'], dtype=object),
 array(['912828ZB9'], dtype=object),
 array(['912810RV2'], dtype=object),
 array(['912828YK0'], dtype=object),
 array(['9128286C9'], dtype=object),
 array(['91282CAV3'], dtype=object),
 array(['912828YE4'], dtype=object),
 array(['91282CAR2'], dtype=object),
 array(['91282CBJ9'], dtype=object),
 array(['912828ZU7'], dtype=object),
 array(['912828YP9'], dtype=object),
 array(['9128287B0'], dtype=object),
 array(['912828ZN3'], dtype=object),
 array(['912828ZG8'], dtype=object),
 array(['91282CBG5'], dtype=object),
 array(['912828YV6'], dtype=object),
 array(['91282CAL5'], dtype=object),
 array(['912810ST6'], dtype=object),
 array(['9128285P1'], dtype=object),
 array(['912810SN9'], dtype=object),
 

In [ ]:
cusip

In [ ]:
pd.to_datetime('2020/11/16').year

In [ ]:
test_bond.cash_flow_table

In [ ]:
range(1)